### Machine Learning using NLP of Wine Descriptions
**Goal:** perform NLP on wine descriptions (using TF-IDF). Use the TF-IDF scores, along with other wine info, to build a machine learning algorithm that can predict the rating score for wine.

**Input:**  A csv file of reviews with descripitons, varietals, region, and score from Wine Enthusiast Magazine. CSV was obtained from use zackthoutt on kaggle.com: [wine review data](https://www.kaggle.com/zynicide/wine-reviews). This dataset is 150,000 wine reviews, and only includes wines with scores from 80-100.

**Expected Output:** A machine learnign algorithm that can take in:
1. Wine description (textual)
2. Country of origin
3. Designation (optional) - this is more specific data about the wine
4. Price
5. Province (optional)
6. Region
7. Varietal
8. Winery

The algorithm should then output a score (from 80-100).

Initially, we will create an algorithm that will attempt to predict score from *only* the text description, and will then compare this to an ML algorithm including text data in addition to the other wine data.

In [ ]:
# import dependencies

import pandas as pd
